In [7]:
from enhance_ocod.labelling.ner_spans import lfs
from enhance_ocod.labelling.weak_labelling import (process_dataframe_batch, create_flat_tag, 
create_commercial_park_tag, remove_overlapping_spans, get_overlap_stats, remove_zero_length_spans, convert_weak_labels_to_standard_format, evaluate_weak_labels)

import pandas as pd 


gt_path = "data/training_data/ground_truth_test_set_labels.csv"

#gt_path = "data/training_data/ground_truth_test_set_labels.csv"

In [8]:
df = pd.read_csv(gt_path)
#df = pd.read_csv("data/OCOD_FULL_2022_02.csv").rename(columns = {'Property Address':'property_address'})
df = df['property_address'].drop_duplicates().reset_index(drop=True).to_frame()


df= create_flat_tag(df)

df = create_commercial_park_tag(df)

/teamspace/studios/this_studio/enhance_ocod/src/enhance_ocod/labelling/weak_labelling.py:273: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['flat_tag'] = df[text_column].str.contains(
/teamspace/studios/this_studio/enhance_ocod/src/enhance_ocod/labelling/weak_labelling.py:281: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['commercial_park_tag'] = df[text_column].str.contains(


In [9]:
test = process_dataframe_batch(df, 
                           batch_size = 1000,
                           text_column = 'property_address',
                           include_function_name = False,
                           save_intermediate = False,
                           output_dir = None,
                           verbose = True)
                           
remove_zero_length_spans(test)
remove_overlapping_spans(test)

Processing batches: 100%|██████████| 1/1 [00:01<00:00,  1.10s/it]


Processing complete:
  - Processed 1000 rows successfully
  - Found 5014 total spans
  - Average 5.01 spans per row


In [10]:
get_overlap_stats(test)

{'total_rows': 1000,
 'rows_with_overlaps': 0,
 'overlap_percentage': 0.0,
 'total_spans_before': 4160,
 'total_overlaps_removed': 0,
 'spans_after': 4160}

In [11]:
test2 = convert_weak_labels_to_standard_format(test)

overall_results, class_df = evaluate_weak_labels(test2, 
    "data/training_data/ner_ready/ground_truth_test_set_labels.json", 
    "enhance_ocod/data/model_performance", 
    'regex_labels')

Evaluating noisy predictions on regex_labels set...
Loaded 1000 ground truth examples
Loaded 1000 predictions

REGEX_LABELS SET RESULTS:
Overall F1: 0.9116
Overall Precision: 0.9340
Overall Recall: 0.8903

Per-class results:
street_number   precision: 0.7947  recall: 0.7186  f1: 0.7547  support: 167
unit_type       precision: 0.4038  recall: 0.6364  f1: 0.4941  support: 33
city            precision: 0.9730  recall: 0.9754  f1: 0.9742  support: 406
unit_id         precision: 0.9688  recall: 0.4079  f1: 0.5741  support: 76
postcode        precision: 1.0000  recall: 1.0000  f1: 1.0000  support: 178
building_name   precision: 0.9096  recall: 0.8728  f1: 0.8909  support: 173
number_filter   precision: 1.0000  recall: 1.0000  f1: 1.0000  support: 17
street_name     precision: 0.9882  recall: 0.9371  f1: 0.9619  support: 445

Results saved:
  Overall: enhance_ocod/data/model_performance/regex_labels_overall_performance.csv
  Per-class: enhance_ocod/data/model_performance/regex_labels_class_pe

In [12]:
import pandas as pd
from enhance_ocod.labelling.weak_labelling import apply_labelling_functions_to_row

# Your example string
text = "15 to 27 (odd numbers) gee street and 66 to 74 (even numbers) bastwick street"

# Create a pandas row (Series) as the function expects
row = pd.Series({'text': text})

# This is how you would call the function
result = apply_labelling_functions_to_row(row, include_function_name=False)


print("Input text:")
print(f"'{text}'")
print("\nLabeled output:")
result

Input text:
'15 to 27 (odd numbers) gee street and 66 to 74 (even numbers) bastwick street'

Labeled output:


{'text': '15 to 27 (odd numbers) gee street and 66 to 74 (even numbers) bastwick street',
 'spans': [{'text': 'odd', 'start': 10, 'end': 13, 'label': 'number_filter'},
  {'text': 'even', 'start': 48, 'end': 52, 'label': 'number_filter'},
  {'text': 'gee street', 'start': 23, 'end': 33, 'label': 'street_name'},
  {'text': 'bastwick street', 'start': 62, 'end': 77, 'label': 'street_name'},
  {'text': '66', 'start': 38, 'end': 40, 'label': 'street_number'},
  {'text': '15', 'start': 0, 'end': 2, 'label': 'street_number'},
  {'text': '15 to 27', 'start': 0, 'end': 8, 'label': 'street_number'},
  {'text': '66 to 74', 'start': 38, 'end': 46, 'label': 'street_number'},
  {'text': '15 to 27', 'start': 0, 'end': 8, 'label': 'unit_id'},
  {'text': '66 to 74', 'start': 38, 'end': 46, 'label': 'unit_id'}]}

In [14]:
df

,Title Number,Tenure,Property Address,District,County,Region,Postcode,Multiple Address Indicator,Price Paid,Proprietor Name (1),...,Proprietor (3) Address (3),Proprietor Name (4),Company Registration No. (4),Proprietorship Category (4),Country Incorporated (4),Proprietor (4) Address (1),Proprietor (4) Address (2),Proprietor (4) Address (3),Date Proprietor Added,Additional Proprietor Indicator
0,MAN296957,Leasehold,"Westleigh Lodge Care Home, Nel Pan Lane, Leigh...",WIGAN,GREATER MANCHESTER,NORTH WEST,WN7 5JT,N,NaN,HC-ONE PROPERTIES 2 LIMITED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,07-08-2017,N
1,MAN297185,Leasehold,"Flat 1, 1a Canal Street, Manchester (M1 3HE)",MANCHESTER,GREATER MANCHESTER,NORTH WEST,M1 3HE,N,270000.0,CANAL STREET INVESTMENTS LIMITED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10-08-2017,N
2,MAN299183,Leasehold,"Flat 201, 1 Regent Road, Manchester (M3 4AY)",MANCHESTER,GREATER MANCHESTER,NORTH WEST,M3 4AY,N,243516.0,WISDOM MAX GROUP LIMITED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13-09-2017,N
3,MAN300821,Freehold,"Land at 2a Gerard Street, Ashton In Makerfield...",WIGAN,GREATER MANCHESTER,NORTH WEST,WN4 9AA,N,NaN,"K/S LONG STREET, MANCHESTER",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10-10-2017,N
4,MAN300971,Leasehold,"Unit 111, Timber Wharf, Worsley Street, Manche...",MANCHESTER,GREATER MANCHESTER,NORTH WEST,M15 4NZ,N,342000.0,URBAN SPLASH WORK LIMITED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11-10-2017,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94086,AGL473126,Leasehold,"155 Bishopsgate, London (EC2M 3AD)",CITY OF LONDON,GREATER LONDON,GREATER LONDON,EC2M 3AD,N,NaN,THE NORINCHUKIN BANK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,08-04-2019,N
94087,AGL473595,Leasehold,"9th and 10th floors, 52 Lime Street, London (E...",CITY OF LONDON,GREATER LONDON,GREATER LONDON,EC3M 7AF,N,NaN,NATIONAL AUSTRALIA BANK LIMITED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11-04-2019,N
94088,AGL475468,Leasehold,"part of Tenth Floor, 6 Bevis Marks, London (EC...",CITY OF LONDON,GREATER LONDON,GREATER LONDON,EC3A 7BA,N,NaN,LARSEN & TOUBRO INFOTECH LTD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02-05-2019,N
94089,AGL476501,Leasehold,"Storage 17, Discovery Dock Apartments East, 3 ...",TOWER HAMLETS,GREATER LONDON,GREATER LONDON,E14 9RU,N,NaN,THE KOMOTO GROUP LIMITED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14-05-2019,N


In [15]:
df = pd.read_csv('data/ocod_history/OCOD_FULL_2022_02.zip').rename(columns = {'Property Address':'property_address'})
#df = pd.read_csv("data/OCOD_FULL_2022_02.csv").rename(columns = {'Property Address':'property_address'})
df = df['property_address'].drop_duplicates().reset_index(drop=True).to_frame()


df= create_flat_tag(df)

df = create_commercial_park_tag(df)

test = process_dataframe_batch(df, 
                           batch_size = 1000,
                           text_column = 'property_address',
                           include_function_name = False,
                           save_intermediate = False,
                           output_dir = None,
                           verbose = True)
                           
remove_zero_length_spans(test)
remove_overlapping_spans(test)

/tmp/ipykernel_286105/2944838506.py:1: DtypeWarning: Columns (24,28,30,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/ocod_history/OCOD_FULL_2022_02.zip').rename(columns = {'Property Address':'property_address'})
/teamspace/studios/this_studio/enhance_ocod/src/enhance_ocod/labelling/weak_labelling.py:273: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['flat_tag'] = df[text_column].str.contains(
/teamspace/studios/this_studio/enhance_ocod/src/enhance_ocod/labelling/weak_labelling.py:281: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['commercial_park_tag'] = df[text_column].str.contains(
Processing batches: 100%|██████████| 93/93 [01:10<00:00,  1.31it/s]



Processing complete:
  - Processed 92361 rows successfully
  - Found 452678 total spans
  - Average 4.90 spans per row


In [17]:
test.to_dataframe()

AttributeError: 'list' object has no attribute 'to_dataframe'